# Implementation Kartentexturen

Für die Implementierung benutzen wir [leafmap](https://leafmap.org/), ein Open-Source-Modul, welches die Benutzung interaktiver Karten innerhalb von Jupyter Umgebungen ermöglicht. [MapLibreGL](https://leafmap.org/maplibre/overview/) erweitert die Funktionalität von leafmap, beispielsweise die [Einblendung von 3D-Gebäuden](https://leafmap.org/maplibre/3d_buildings/). In dieser Anleitung zeigen wir die [Hervorhebung von Gebieten](https://leafmap.org/maplibre/geojson_polygon/), die im .geojson-Format begrenzt werden.

In [1]:
import leafmap.maplibregl as leafmap
import json

Wir erstellen eine leafmap-Karte, die auf Deutschland zentriert ist.

In [2]:
m = leafmap.Map(zoom=5, center=[10.4515,51.1657], style="liberty")

Jetzt öffnen wir eine [.geojson-Datei](https://raw.githubusercontent.com/isellsoap/deutschlandGeoJSON/refs/heads/main/2_bundeslaender/2_hoch.geo.json), die die Formen der Bundesländer enthält. Anschließend definieren wir ein dict mit den IDs der Bundesländer und dem Verweis auf die jeweilige Textur. Bislang kann MapLibre noch nicht mit SVG-Dateien als Textur arbeiten.

In [3]:
with open("germany_states.geojson", "r") as f:
    geojson_data = json.load(f)

pattern_mappings = {
    "DE-BW": "caretpattern.png",     # Baden-Württemberg
    "DE-BY": "diamondpattern.png",   # Bayern
    "DE-BE": "dotspattern.png",      # Berlin
    "DE-BB": "tristarpattern.png",   # Brandenburg
    "DE-HB": "spiralpattern.png",    # Bremen
    "DE-HH": "squarepattern.png",    # Hamburg
    "DE-HE": "spiralpattern.png",    # Hessen
    "DE-MV": "spiralpattern.png",    # Mecklenburg-Vorpommern
    "DE-NI": "crosspattern.png",     # Niedersachsen
    "DE-NW": "diamondpattern.png",   # Nordrhein-Westfalen
    "DE-RP": "trianglepattern.png",  # Rheinland-Pfalz
    "DE-SL": "crosspattern.png",     # Saarland
    "DE-SN": "stripepattern.png",    # Sachsen
    "DE-ST": "squarepattern.png",    # Sachsen-Anhalt
    "DE-SH": "tristarpattern.png",   # Schleswig-Holstein
    "DE-TH": "trianglepattern.png",  # Thüringen
}

Wir öffnen alle Texturdateien und verwenden die Funktion `add_image`, um die Dateien in leafmap bereitzustellen. Mit `add_source` werden die Bundesländer als Quellen geladen und mittels `add_layer` als Ebene mit Textur der Karte hinzugefügt.

In [4]:
# Alle Muster laden
for state_id, pattern_file in pattern_mappings.items():
    pattern_id = f"pattern-{state_id}"
    pattern_url = f"../../_static/pattern/png/{pattern_file}"
    m.add_image(pattern_id, pattern_url)

# Jedes Bundesland als Quelle bzw. Ebene hinzufügen
for feature in geojson_data["features"]:
    state_id = feature["properties"]["id"]
    state_name = feature["properties"]["name"]
    
    source_id = f"source-{state_id}"
    source = {
        "type": "geojson",
        "data": feature
    }
    m.add_source(source_id, source)
    
    layer_id = f"layer-{state_id}"
    layer = {
        "id": layer_id,
        "type": "fill",
        "source": source_id,
        "paint": {"fill-pattern": f"pattern-{state_id}"}
    }
    m.add_layer(layer, 'label_other')
    
    # Umriss der Bundesländer hinzufügen
    outline_layer_id = f"outline-{state_id}"
    outline_layer = {
        "id": outline_layer_id,
        "type": "line",
        "source": source_id,
        "paint": {
            "line-color": "#000000",
            "line-width": 1
        }
    }
    m.add_layer(outline_layer, 'label_other')

Um die Karte anzuzeigen geben wir den Namen an:

In [5]:
m

Map(height='600px', map_options={'bearing': 0, 'center': (10.4515, 51.1657), 'pitch': 0, 'style': 'https://til…